In [1]:
import statsmodels
from statsmodels import stats
from statsmodels.stats import weightstats
from scipy.stats import t,norm
import numpy as np
import pandas as pd
import  statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats import proportion
from scipy.stats import chisquare,chi2_contingency
import scipy
from scipy import stats
import pylab
from pylab import plot,show,legend,title,xlabel,ylabel
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel('CDAC_DataBook.xlsx',sheet_name = "salaries")

In [5]:
df.head()

,rank,discipline,yrs_phd,yrs_service,gender,salary
0,Prof,B,19,18,Male,139750
1,Prof,B,20,16,Male,173200
2,AsstProf,B,4,3,Male,79750
3,Prof,B,45,39,Male,115000
4,Prof,B,40,41,Male,141500


In [3]:
df = df[['rank','yrs_service','salary']]

In [8]:
df.head()

,rank,yrs_service,salary
0,Prof,18,139750
1,Prof,16,173200
2,AsstProf,3,79750
3,Prof,39,115000
4,Prof,41,141500


In [4]:
rank_dummy = pd.get_dummies(df['rank'],drop_first=True).astype(int)

In [13]:
rank_dummy

,AsstProf,Prof
0,0,1
1,0,1
2,1,0
3,0,1
4,0,1
...,...,...
392,0,1
393,0,1
394,0,1
395,0,1


In [5]:
df = df.drop('rank',axis=1)

In [6]:
df = pd.concat([df,rank_dummy],axis=1)

In [17]:
## here AssoProf has been taken as reference and at situations where AsstProf and Prof are 0 those are the locations of AssoProf.
df.head()

,yrs_service,salary,AsstProf,Prof
0,18,139750,0,1
1,16,173200,0,1
2,3,79750,1,0
3,39,115000,0,1
4,41,141500,0,1


In [7]:
x_train,x_test,y_train,y_test=train_test_split(df.drop('salary',axis=1),df.salary, test_size=0.2)
x_train = sm.add_constant(x_train,prepend = False)
x_test = sm.add_constant(x_test,prepend=False)
mod1 = sm.OLS(y_train,x_train).fit()
print(mod1.summary())

                            OLS Regression Results                            
Dep. Variable:                 salary   R-squared:                       0.405
Model:                            OLS   Adj. R-squared:                  0.400
Method:                 Least Squares   F-statistic:                     71.13
Date:                Sun, 09 Jun 2024   Prob (F-statistic):           4.20e-35
Time:                        21:57:40   Log-Likelihood:                -3644.0
No. Observations:                 317   AIC:                             7296.
Df Residuals:                     313   BIC:                             7311.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
yrs_service  -164.3126    130.198     -1.262      

In [19]:
y_pred = mod1.predict(x_test)

In [20]:
x_test.head()

,yrs_service,AsstProf,Prof,const
377,3,1,0,1.0
365,40,0,1,1.0
239,6,0,1,1.0
330,60,0,1,1.0
146,4,1,0,1.0


In [21]:
y_pred.head()

377     80194.956707
365    122714.621514
239    127715.100394
330    119773.163350
146     80047.883798
dtype: float64

In [22]:
y_test.head()

377     77081
365    101036
239     96200
330    192253
146     92000
Name: salary, dtype: int64

In [8]:
mydata = pd.DataFrame([[5,0,0,1],[5,1,0,1],[5,0,1,1]])
mydata.columns = x_test.columns
mypred = mod1.predict(mydata)

In [9]:
print(mypred)

0     93930.008518
1     80269.967783
2    130878.699206
dtype: float64


In [12]:
## matching with above mypred data and using the salary(resultant) equation we derived 
salary  = -164.3126*5+9.475e+04+3.695e+04
salary

130878.437

In [33]:
## Now ensuring the data from yrs of service 

ydata = pd.DataFrame([[5,0,0,1],[8,0,0,1],[10,0,0,1]])
ydata.columns = x_test.columns
mypred = mod1.predict(ydata)
mypred

0    93690.801616
1    93249.582891
2    92955.437075
dtype: float64

In [36]:
#substracting change of years of service for 3 years (5 to 8 )
93249.582891-93690.801616

-441.2187249999988

In [37]:
#now for each year it would be 
-441.2187249999988/3
#this value is same with what the model summarised as coeffecient for yrs of service

-147.07290833333295

# Start of Linear Regression
## Binary Linear Regression

In [13]:
df = pd.read_excel('CDAC_DataBook.xlsx',sheet_name='mtcars')
df.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [14]:
df=df[['hp','wt','am']]

In [15]:
df.head()

,hp,wt,am
0,110,2.620,1
1,110,2.875,1
2,93,2.320,1
3,110,3.215,0
4,175,3.440,0


In [17]:
x_train = df.iloc[:28]
x_test = df.iloc[28:]

y_train = x_train.am
y_test=x_test.am
x_train = x_train.drop('am',axis=1)
x_test = x_test.drop('am',axis=1)

x_train = sm.add_constant(x_train,prepend=False)
x_test = sm.add_constant(x_test,prepend=False)

In [18]:
mod2 = sm.Logit(y_train,x_train).fit()
print(mod2.summary())

Optimization terminated successfully.
         Current function value: 0.136274
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                     am   No. Observations:                   28
Model:                          Logit   Df Residuals:                       25
Method:                           MLE   Df Model:                            2
Date:                Fri, 07 Jun 2024   Pseudo R-squ.:                  0.7830
Time:                        13:00:46   Log-Likelihood:                -3.8157
converged:                       True   LL-Null:                       -17.582
Covariance Type:            nonrobust   LLR p-value:                 1.050e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
hp             0.0107      0.039      0.273      0.785      -0.066       0.087
wt            -6.1413      3.

In [19]:
y_pred = mod2.predict(x_test)

In [20]:
y_pred.head()

28    0.286330
29    0.644190
30    0.068357
31    0.457093
dtype: float64

## Trial for depicting log values

In [23]:
np.log(4) #log of a number greater than 1 is always +ve >0

1.3862943611198906

In [24]:
np.log(1/4) #log of a number less than 1 is always -ve <0

-1.3862943611198906

## Now depicting how much weight affects transmission as per our model

In [26]:
x_train.head()

,hp,wt,const
0,110,2.620,1.0
1,110,2.875,1.0
2,93,2.320,1.0
3,110,3.215,1.0
4,175,3.440,1.0


In [27]:
mydata = pd.DataFrame([[110,2,1],[110,2.5,1],[110,3,1]])
mydata.columns = x_train.columns

In [29]:
preds = mod2.predict(mydata)

In [32]:
#here as weight increases probability of predictor value (here it is automatic which is depicted by 1) will decrease
# which can also be stated as when weight increase probability of reference value(which would be manual depicted by 0) decreases
preds

0    0.990328
1    0.826093
2    0.180573
dtype: float64

In [37]:
#log of odds for 2 { odd ratios for taking place = p/(1-p) }
lo2 = np.log(0.990328/(1-0.990328))
lo2

4.62880108807538

In [36]:
lo25 = np.log(0.826093/(1-0.826093))
lo25

1.5581866844130583

In [40]:
# we are dividing by 0.5 because the difference of wieghts here is 0.5 and 
# we are trying to find the difference of change of log of odds for a unit weight 
(lo25-lo2)/0.5

-6.1412288073246435

In [2]:
# this (-6.1412288073246435) is the change in the log of odds ratio for automatic transmission (1) when the weight increase by a unit (1) tonne
# change of log of odds gives us the order/degree of impact a factor/predictor has on an event/response occuring
# as the magnitude of the difference increases  the impact also increases
# if this change is equal then probability of both the events occuring is 0 then null hypothesis would not get rejected
#extra
# log x = y
# e**x = y

# log(p1/(1-p1)) - log(p2/(1-p2))
# If the prob of auto at wt 2 is 0.8
# find the prob of auto at wt 2.5